## Chuẩn bị dữ liệu ASR với bộ 28k Vietnamese Voice

Notebook này thực hiện các bước:

1. Tải bộ dữ liệu `natmin322/28k_vietnamese_voice_augmented_of_VigBigData` từ Hugging Face.
2. Chuẩn hóa cấu trúc dữ liệu cho bài toán ASR:
   - Chuẩn audio về sampling rate 16 kHz, mono (dùng cột `audio` của Hugging Face Datasets).
   - Chuẩn hóa trường transcript (cột `sentence`).
3. Gộp các split train (`train_1` → `train_5`) thành một tập train lớn.
4. Chia lại thành `train` và `validation` (90% / 10%).
5. Giữ nguyên split `test` gốc của dataset.
6. Lưu lại các split dưới dạng Hugging Face Dataset bằng `save_to_disk` để dùng cho huấn luyện ASR sau này.


In [ ]:
# Cài đặt thư viện cần thiết (chạy một lần trong môi trường mới)
# Nếu đã có rồi thì có thể bỏ qua cell này
%pip install -q datasets soundfile librosa tqdm

from datasets import load_dataset, concatenate_datasets, Audio
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import json

# Thư mục lưu dataset đã xử lý
OUTPUT_DIR = Path("./data/asr_28k")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("Thư mục lưu dữ liệu:", OUTPUT_DIR.resolve())
a
# 1. Tải dataset 28k từ Hugging Face
print("\n[1/4] Đang tải dataset 28k Vietnamese Voice từ Hugging Face...")
vig_dataset = load_dataset("natmin322/28k_vietnamese_voice_augmented_of_VigBigData")
print("Các splits có sẵn:", list(vig_dataset.keys()))

# 2. Gộp các split train_1 → train_5
print("\n[2/4] Đang gộp các split train_1 → train_5...")
train_splits = [f"train_{i}" for i in range(1, 6)]
train_datasets = [vig_dataset[split] for split in train_splits]

vig_train_all = concatenate_datasets(train_datasets)
vig_test = vig_dataset["test"]

print(f"Số mẫu train (gộp): {len(vig_train_all)}")
print(f"Số mẫu test: {len(vig_test)}")

# 3. Chuẩn hóa cấu trúc cho ASR: audio (16 kHz), transcript (lowercase, strip)
print("\n[3/4] Đang chuẩn hóa cấu trúc dữ liệu cho ASR...")

def prepare_asr_example(example):
    """Chuẩn hóa 1 sample cho bài toán ASR."""
    # Cột `audio` của Hugging Face đã chứa array và sampling_rate
    audio = example["audio"]
    transcript = example["sentence"]

    return {
        "audio": audio,
        "transcript": str(transcript).lower().strip(),
        "duration": float(audio["duration"]),
    }

# Áp dụng cho train và test
vig_train_processed = vig_train_all.map(
    prepare_asr_example,
    remove_columns=[col for col in vig_train_all.column_names if col not in ["audio", "sentence"]],
    num_proc=4,
)

vig_test_processed = vig_test.map(
    prepare_asr_example,
    remove_columns=[col for col in vig_test.column_names if col not in ["audio", "sentence"]],
    num_proc=4,
)

# Đảm bảo cột audio được cast về Audio với sampling_rate = 16 kHz
vig_train_processed = vig_train_processed.cast_column("audio", Audio(sampling_rate=16000))
vig_test_processed = vig_test_processed.cast_column("audio", Audio(sampling_rate=16000))

print("Hoàn tất chuẩn hóa. Ví dụ 1 sample train:")
print({k: vig_train_processed[0][k] for k in ["transcript", "duration"]})

# 4. Chia train thành train/validation (90% / 10%)
print("\n[4/4] Đang chia train thành train/validation (90% / 10%)...")

split_dict = vig_train_processed.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dict["train"]
val_dataset = split_dict["test"]

total_train = len(train_dataset)
total_val = len(val_dataset)
total_test = len(vig_test_processed)

print(f"Số mẫu train: {total_train}")
print(f"Số mẫu val:   {total_val}")
print(f"Số mẫu test:  {total_test}")

# 5. Lưu dataset đã xử lý xuống đĩa bằng save_to_disk
print("\nĐang lưu các split đã xử lý xuống đĩa...")

train_path = OUTPUT_DIR / "train"
val_path = OUTPUT_DIR / "val"
test_path = OUTPUT_DIR / "test"

train_dataset.save_to_disk(str(train_path))
val_dataset.save_to_disk(str(val_path))
vig_test_processed.save_to_disk(str(test_path))

print("Đã lưu:")
print("  - Train:", train_path.resolve())
print("  - Val:  ", val_path.resolve())
print("  - Test: ", test_path.resolve())

# 6. Lưu thống kê cơ bản vào file JSON
print("\nĐang ghi thống kê dữ liệu vào stats.json...")

train_transcripts = [ex["transcript"] for ex in train_dataset]
train_lengths = [len(t) for t in train_transcripts]
train_durations = [ex["duration"] for ex in train_dataset]

stats = {
    "train_samples": total_train,
    "val_samples": total_val,
    "test_samples": total_test,
    "avg_transcript_length": float(sum(train_lengths) / len(train_lengths)),
    "min_transcript_length": int(min(train_lengths)),
    "max_transcript_length": int(max(train_lengths)),
    "avg_duration": float(sum(train_durations) / len(train_durations)),
    "min_duration": float(min(train_durations)),
    "max_duration": float(max(train_durations)),
}

with open(OUTPUT_DIR / "stats.json", "w", encoding="utf-8") as f:
    json.dump(stats, f, ensure_ascii=False, indent=2)

print("Hoàn thành chuẩn bị dữ liệu ASR cho bộ 28k Vietnamese Voice.")
print("Thống kê chính:")
for k, v in stats.items():
    print(f"  {k}: {v}")


^C
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'datasets'

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tokenizers 0.14.1 requires huggingface_hub<0.18,>=0.16.4, but you have huggingface-hub 1.1.5 which is incompatible.
